In [1]:
import pandas as pd

from autogluon.tabular import TabularPredictor
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('../../data/tlp/synthetic-tvaes.csv')

In [3]:
df.head()

,demo-genero,demo-fechaEvaluacion,demo-fechaNacimiento,demo-rangoEdad,demo-edad,clin-reservaCognitiva_total,clin-reservaCognitiva_idiomas,clin-reservaCognitiva_ocupacion,clin-reservaCognitiva_escolaridad,clin-ansiedad,...,eval-TLP-Tavec-25_Discriminabilidad-PD,eval-TLP-Tavec-26_Sesgo-PD,eval-TLP-Tavec-27_RI_B_frente_RI_A1-PD,eval-TLP-Tavec-28_RL_CP_frente_RI_A5-PD,eval-TLP-Tavec-29_RCl_Cp_frente_RCl_LP-PD,eval-TLP-Tavec-30_RL_LP_frente_RL_CP-PD,eval-TLP-Tavec-31_RCl_LP_frente_RL_LP-PD,eval-TLP-Tavec-32_Recon_Ac_frente_RL_LP-PD,eval-TLP-Tavec-33_Recon_Ac_frente_RCl_LP-PD,ED_2Clases
0,2,2024-02-22T00:00:00,1942-05-08T00:00:00,Rango3,81,12,0,1,3,False,...,84.09,0.14,-20.00,0.00,33.33,37.50,-33.33,-38.46,-7.69,D
1,2,2024-06-26T00:00:00,1948-10-07T00:00:00,Rango3,75,8,0,1,3,True,...,79.55,-0.33,-71.43,12.50,0.00,0.00,0.00,-20.00,-20.00,D
2,2,2024-05-14T00:00:00,1952-05-08T00:00:00,Rango2,72,11,0,0,4,False,...,88.64,-0.60,-14.29,75.00,25.00,-20.00,0.00,-16.67,-16.67,D
3,2,2024-04-10T00:00:00,1964-01-27T00:00:00,Rango1,60,18,2,1,4,False,...,90.91,0.60,60.00,16.67,0.00,50.00,-20.00,-50.00,-37.50,D
4,2,2024-05-14T00:00:00,1959-09-16T00:00:00,Rango1,64,13,0,1,4,False,...,86.36,-0.71,0.00,66.67,0.00,-33.33,0.00,-10.00,-10.00,D


In [4]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train.shape, df_test.shape

((396, 80), (100, 80))

In [5]:
predictor= TabularPredictor(
    label='ED_2Clases',
    problem_type='binary',
    eval_metric='roc_auc',
    sample_weight='balance_weight',
    path='AutogluonModels/synthetic/tvaes/v2'
    )

In [6]:
predictor.fit(
    train_data = df_train,
    presets = ['high_quality'],
    time_limit = 1 * 3600,
    auto_stack = True,
    excluded_model_types=['KNN','RF','XT', 'LR'],
    verbosity = 2
)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.2
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          12
Memory Avail:       10.70 GB / 15.94 GB (67.2%)
Disk Space Avail:   105.27 GB / 446.36 GB (23.6%)
Presets specified: ['high_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Note: `save_bag_folds=False`! This will greatly reduce peak disk usage during fit (by ~8x), but runs the risk of an out-of-memory error during model refit if memory is small relative to the data size.
	You can avoid this risk by setting `save_bag_folds=True`.
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting an

(_ray_fit pid=5332) [1000]	valid_set's binary_logloss: 0.102873


(_dystack pid=18088) 	0.9709	 = Validation score   (roc_auc)
(_dystack pid=18088) 	2.3s	 = Training   runtime
(_dystack pid=18088) 	0.04s	 = Validation runtime
(_dystack pid=18088) Fitting model: NeuralNetFastAI_r191_BAG_L1 ... Training model for up to 334.83s of the 632.86s of remaining time.
(_dystack pid=18088) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=0.04%)
(_ray_fit pid=11128) No improvement since epoch 0: early stopping
(_dystack pid=18088) 	0.8937	 = Validation score   (roc_auc)
(_dystack pid=18088) 	6.22s	 = Training   runtime
(_dystack pid=18088) 	0.09s	 = Validation runtime
(_dystack pid=18088) Fitting model: CatBoost_r9_BAG_L1 ... Training model for up to 325.39s of the 623.41s of remaining time.
(_dystack pid=18088) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=3.63%)
(_dystack pid=18088) 	0.9688	 = Validation s

(_ray_fit pid=6784) [1000]	valid_set's binary_logloss: 0.29819 [repeated 2x across cluster]


(_dystack pid=18088) 	0.9653	 = Validation score   (roc_auc)
(_dystack pid=18088) 	5.44s	 = Training   runtime
(_dystack pid=18088) 	0.06s	 = Validation runtime
(_dystack pid=18088) Fitting model: NeuralNetTorch_r22_BAG_L1 ... Training model for up to 166.55s of the 464.57s of remaining time.
(_dystack pid=18088) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=0.02%)
(_dystack pid=18088) 	Warning: Exception caused NeuralNetTorch_r22_BAG_L1 to fail during training... Skipping this model.
(_dystack pid=18088) 		ray::_ray_fit() (pid=19808, ip=127.0.0.1)
(_dystack pid=18088)   File "python\ray\_raylet.pyx", line 1889, in ray._raylet.execute_task
(_dystack pid=18088)   File "c:\Users\jgala\uned\tfm\synthetic-data\.venv\lib\site-packages\autogluon\core\models\ensemble\fold_fitting_strategy.py", line 413, in _ray_fit
(_dystack pid=18088)     fold_model.fit(X=X_fold, y=y_fold, X_val=X_val_fold, y_val=y_val_fold, time

In [7]:
predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                                model  score_val eval_metric  pred_time_val     fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0                 WeightedEnsemble_L2   0.977686     roc_auc       0.207192    41.759184                0.001023           0.033920            2      False         59
1                 CatBoost_r69_BAG_L1   0.975975     roc_auc       0.063878    29.564252                0.063878          29.564252            1      False         23
2                   LightGBMXT_BAG_L1   0.975899     roc_auc       0.035680     1.250397                0.035680           1.250397            1      False          1
3                 LightGBM_r30_BAG_L1   0.974239     roc_auc       0.069385     3.286644                0.069385           3.286644            1      False         44
4                LightGBM_r188_BAG_L1   0.974188     roc_auc       0.035332     2.354293               

{'model_types': {'LightGBMXT_BAG_L1': 'StackerEnsembleModel_LGB',
  'LightGBM_BAG_L1': 'StackerEnsembleModel_LGB',
  'CatBoost_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'NeuralNetFastAI_BAG_L1': 'StackerEnsembleModel_NNFastAiTabular',
  'XGBoost_BAG_L1': 'StackerEnsembleModel_XGBoost',
  'LightGBMLarge_BAG_L1': 'StackerEnsembleModel_LGB',
  'CatBoost_r177_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'LightGBM_r131_BAG_L1': 'StackerEnsembleModel_LGB',
  'NeuralNetFastAI_r191_BAG_L1': 'StackerEnsembleModel_NNFastAiTabular',
  'CatBoost_r9_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'LightGBM_r96_BAG_L1': 'StackerEnsembleModel_LGB',
  'XGBoost_r33_BAG_L1': 'StackerEnsembleModel_XGBoost',
  'CatBoost_r137_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'NeuralNetFastAI_r102_BAG_L1': 'StackerEnsembleModel_NNFastAiTabular',
  'CatBoost_r13_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'LightGBM_r188_BAG_L1': 'StackerEnsembleModel_LGB',
  'NeuralNetFastAI_r145_BAG_L1': 'StackerEnsembleModel_NNFast

In [8]:
auxiliary_metrics = ['balanced_accuracy', 'f1', 'f1_macro', 'f1_micro', 'roc_auc', 'average_precision', 'precision', 'recall', 'log_loss', 'pac_score']

In [9]:
predictor.evaluate(
    df_test,
    silent = False,
    auxiliary_metrics = auxiliary_metrics
    )

Evaluation: roc_auc on test data: 0.987012987012987
Evaluations on test data:
{
    "roc_auc": 0.987012987012987,
    "accuracy": 0.94,
    "balanced_accuracy": 0.9391233766233766,
    "mcc": 0.8782467532467533,
    "f1": 0.9464285714285714,
    "precision": 0.9464285714285714,
    "recall": 0.9464285714285714
}


{'roc_auc': 0.987012987012987,
 'accuracy': 0.94,
 'balanced_accuracy': 0.9391233766233766,
 'mcc': 0.8782467532467533,
 'f1': 0.9464285714285714,
 'precision': 0.9464285714285714,
 'recall': 0.9464285714285714}

In [24]:
predictor.model_best

'WeightedEnsemble_L2_FULL'

In [17]:
predictors = predictor.leaderboard(
    df_test,
    extra_metrics = auxiliary_metrics,
    extra_info=True,
    silent=True
)

In [25]:
predictors

,model,score_test,balanced_accuracy,f1,f1_macro,f1_micro,roc_auc,average_precision,precision,recall,...,hyperparameters,hyperparameters_fit,ag_args_fit,features,compile_time,child_hyperparameters,child_hyperparameters_fit,child_ag_args_fit,ancestors,descendants
0,LightGBM_r143_BAG_L1_FULL,0.992289,0.939123,0.946429,0.939123,0.94,0.992289,0.994303,0.946429,0.946429,...,"{'use_orig_features': True, 'valid_stacker': True, 'max_base_models': 0, 'max_base_models_per_type': 'auto', 'save_bag_folds': True}",{},"{'max_memory_usage_ratio': 1.15, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': None, 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'temperature_scalar': None, 'drop_unique': False}","[eval-TLP-Tavec-14_ESem_RL_CP-PD, clin-Hipercolesterolemia, clin-reservaCognitiva_idiomas, clin-reservaCognitiva_escolaridad, ques-MMSE-fijacion-PD, eval-TLP-FigRey-totalMemoria-PD, clin-tipoAlcohol.capital_ratio, clin-Hipertension, eval-TLP-Tavec-25_Discriminabilidad-PD, eval-TLP-Tavec-2_RI_A5-PD, ques-Sus-total-PD, ques-NavEspacial-resultadosWFQ_SA-PD, clin-tipoAlcohol.symbol_ratio. , eval-TLP-Tavec-20_P-PD, demo-fechaNacimiento.day, eval-TLP-Tavec-28_RL_CP_frente_RI_A5-PD, ques-QuejasMemo-Total-PD, demo-rangoEdad, eval-TLP-Tavec-4_RI_B-PD, ques-MMSE-escolaridad-PD, eval-TLP-FigRey-Durac...",None,"{'learning_rate': 0.01343464462043561, 'extra_trees': False, 'feature_fraction': 0.9408897917880529, 'min_data_in_leaf': 21, 'num_leaves': 178, 'num_boost_round': 472}",{'num_boost_round': 472},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': ['bool', 'int', 'float', 'category'], 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'temperature_scalar': None}",[],[]
1,LightGBM_BAG_L1_FULL,0.991883,0.948052,0.955752,0.949140,0.95,0.991883,0.993962,0.947368,0.964286,...,"{'use_orig_features': True, 'valid_stacker': True, 'max_base_models': 0, 'max_base_models_per_type': 'auto', 'save_bag_folds': True}",{},"{'max_memory_usage_ratio': 1.15, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': None, 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'temperature_scalar': None, 'drop_unique': False}","[eval-TLP-Tavec-14_ESem_RL_CP-PD, clin-Hipercolesterolemia, clin-reservaCognitiva_idiomas, clin-reservaCognitiva_escolaridad, ques-MMSE-fijacion-PD, eval-TLP-FigRey-totalMemoria-PD, clin-tipoAlcohol.capital_ratio, clin-Hipertension, eval-TLP-Tavec-25_Discriminabilidad-PD, eval-TLP-Tavec-2_RI_A5-PD, ques-Sus-total-PD, ques-NavEspacial-resultadosWFQ_SA-PD, clin-tipoAlcohol.symbol_ratio. , eval-TLP-Tavec-20_P-PD, demo-fechaNacimiento.day, eval-TLP-Tavec-28_RL_CP_frente_RI_A5-PD, ques-QuejasMemo-Total-PD, demo-rangoEdad, eval-TLP-Tavec-4_RI_B-PD, ques-MMSE-escolaridad-PD, eval-TLP-FigRey-Durac...",None,"{'learning_rate': 0.05, 'num_boost_round': 122}",{'num_boost_round': 122},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': ['bool', 'int', 'float', 'category'], 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'temperature_scalar': None}",[],[]
2,CatBoost_r13_BAG_L1_FULL,0.989854,0.935065,0.934579,0.929655,0.93,0.989854,0.992096,0.980392,0.892857,...,"{'use_orig_features': True, 'valid_stacker': True, 'max_base_models': 0, 'max_base_models_per_type': 'auto', 'save_bag_folds':

In [26]:
predictors.to_excel(
    '../../data/tlp/predictors.xlsx',
    index=False
)

In [11]:
predictor.feature_importance(data=df_train)

These features in provided data are not utilized by the predictor and will be ignored: ['clin-Covid_sintomaSarpullido']
Computing feature importance via permutation shuffling for 78 features using 396 rows with 5 shuffle sets...
	46.52s	= Expected runtime (9.3s per shuffle set)
	17.61s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
eval-TLP-Stroop-color-PD,0.004131,0.001049,0.000459,5,0.006291,0.001971
demo-genero,0.000000,0.000000,0.500000,5,0.000000,0.000000
eval-TLP-Tavec-5_Rg_Pr-PD,0.000000,0.000000,0.500000,5,0.000000,0.000000
eval-TLP-Tavec-12_ESem_RI_A-PD,0.000000,0.000000,0.500000,5,0.000000,0.000000
eval-TLP-Tavec-11_RCl_LP-PD,0.000000,0.000000,0.500000,5,0.000000,0.000000
...,...,...,...,...,...,...
ques-Sus-totalPar-PD,0.000000,0.000000,0.500000,5,0.000000,0.000000
ques-NavEspacial-resultadosWFQ_DE-PD,0.000000,0.000000,0.500000,5,0.000000,0.000000
ques-NavEspacial-resultadosWFQ_SA-PD,0.000000,0.000000,0.500000,5,0.000000,0.000000
ques-NavEspacial-resultadosWFQ_NO-PD,0.000000,0.000000,0.500000,5,0.000000,0.000000
